<a href="https://colab.research.google.com/github/No-Qubit-Left-Behind/Control-Engineering-in-TF/blob/master/TF_Propagator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Gradients of a propagor

In [0]:
from __future__ import absolute_import, division, print_function, unicode_literals
from google.colab import files
%tensorflow_version 2.x
#!pip install tensorflow==2.0.0-beta0
import tensorflow as tf
import numpy as np
import time
print(tf.__version__)

2.1.0


In [0]:
uploaded = files.upload()

KeyboardInterrupt: ignored

In [0]:
uploaded = files.upload()

Saving Rabi freq list.dat to Rabi freq list.dat


In [0]:
class Propagator(object):
  def __init__(self):
    self.length = 1372707
    self.steps_power = int(np.floor(np.log2(self.length)))
    self.bool_array = self.gen_bool_array()
    self.dim = 2
    x = -0.0602715189469 * tf.constant(
        [[0 + 0j, 0 + 1j], [0 + 1j, 0 + 0j]], dtype=tf.complex128
    )
    y = -0.0602715189469 * tf.constant(
        [[0, 1], [-1, 0]], dtype=tf.complex128
    )
    z = -0.0602715189469 * tf.constant(
        [[0 + 1j, 0 + 0j], [0 + 0j, 0 - 1j]], dtype=tf.complex128
    )
    self.Rabi_strength = tf.Variable(0., dtype=tf.complex128)
    self.resonance_offset = tf.Variable(0., dtype=tf.complex128)
    self.generators =  tf.stack([x, y, z])
    self.controls = tf.convert_to_tensor(
        np.genfromtxt('10 kHz Comb Pulse Waveform Shifted.dat'),
        dtype=tf.complex128
    )

  def gen_bool_array(self):
    length = self.length
    bool_array = []
    for i in range(self.steps_power):
      bool_array.append(bool(np.mod(length, 2)))
      length = np.floor(length / 2)
    return bool_array
  
  @tf.function
  def exponents(self):
    offset_vector = tf.ones(
        (self.length, 1), dtype=tf.complex128
    )
    exponents = tf.linalg.tensordot(
        tf.concat([
                   self.Rabi_strength * self.controls,
                   self.resonance_offset * offset_vector
                   ], axis=1),
        self.generators,
        1
    )
    return exponents
  
  @tf.function
  def exponentials(self):
    exponents = self.exponents()
    return tf.linalg.expm(exponents)
  
  @tf.function
  def propagate_exps(self):
    step_exps = self.exponentials()
    for i in range(self.steps_power):
      if self.bool_array[i]:
        odd_exp = step_exps[-1, :, :]
        step_exps = tf.linalg.matmul(
          step_exps[1::2, :, :],
          step_exps[0:-1, :, :][0::2, :, :]
        )
        step_exps = tf.concat([
            step_exps[0:-1, :, :],
            [tf.linalg.matmul(odd_exp ,step_exps[-1, :, :])]
        ], 0)
      else:
        step_exps = tf.linalg.matmul(
          step_exps[1::2, :, :],
          step_exps[0::2, :, :]
        )
    return tf.squeeze(step_exps)
    
  @tf.function
  def __call__(self):
    z = tf.constant([[1., 0.], [0., -1.]], dtype=tf.complex128)
    propagator = self.propagate_exps()
    z_times_propagator = tf.linalg.matmul(z, propagator)
    z_times_propagator_adjoint = tf.linalg.matmul(
        z, tf.linalg.adjoint(propagator)
    )
    return 0.5 * tf.math.real(
        tf.linalg.trace(
            tf.linalg.matmul(
                z_times_propagator,
                z_times_propagator_adjoint
            )
        )
    )

propagator = Propagator()
Rabi_strengths = np.genfromtxt('Rabi freq list.dat')
print('Start...')

expectations = np.array([[0., 0.]])
propagator.resonance_offset.assign(-0.001)

for strength in np.nditer(Rabi_strengths):
    propagator.Rabi_strength.assign(strength)
    expectation_value = propagator().numpy()
    expectations = np.append(expectations, [[strength, expectation_value]], axis=0)
    print(strength)

np.savetxt('magnetization_N1partial.dat', expectations[1:])
files.download('magnetization_N1partial.dat')
print('N1 done...')

Start...
0.42
0.4205
0.421
0.4215
0.422
0.4225
0.423
0.4235
0.424
0.4245
0.425
0.4255
0.426
0.4265
0.427
0.4275
0.428
0.4285
0.429
0.4295
0.43
0.4305
0.431
0.4315
0.432
0.4325
0.433
0.4335
0.434
0.4345
0.435
0.4355
0.436
0.4365
0.437
0.4375
0.438
0.4385
0.439
0.4395
0.44
0.4405
0.441
0.4415
0.442
0.4425
0.443
0.4435
0.444
0.4445
0.445
0.4455
0.446
0.4465
0.447
0.4475
0.448
0.4485
0.449
0.4495
0.44999999999999996
0.4505
0.45099999999999996
0.4515
0.45199999999999996
0.4525
0.45299999999999996
0.4535
0.45399999999999996
0.4545
0.45499999999999996
0.4555
0.45599999999999996
0.45649999999999996
0.45699999999999996
0.45749999999999996
0.45799999999999996
0.45849999999999996
0.45899999999999996
0.45949999999999996
0.45999999999999996
0.46049999999999996
0.46099999999999997
0.46149999999999997
0.46199999999999997
0.46249999999999997
0.46299999999999997
0.46349999999999997
0.46399999999999997
0.46449999999999997
0.46499999999999997
0.46549999999999997
0.46599999999999997
0.46649999999999997
0.

MessageError: ignored